In [1]:
%pylab inline
import pandas as pd
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import log_loss

Populating the interactive namespace from numpy and matplotlib


In [2]:
def plot_roc(y_test, preds):
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()


In [3]:
files=!find results/ -name \*.txt |grep data9

In [4]:
files

['results/Capsule-Forensics-v2/data9.txt',
 'results/ClassNSeg/data9.txt',
 'results/MesoNet/data9.txt',
 'results/CNNDetection/data9.txt',
 'results/adversarial_deepfakes/data9.txt']

In [3]:
real_datasets=['celeba','ffhq']
pairs=[
    ('celeba','stargan'),
    ('celeba','attgan'),
    ('celeba','gdwct'),
    ('ffhq','stylegan'),
    ('ffhq','stylegan2'),
           ]

In [4]:
for k,v in pairs:
    print(k,v)

celeba stargan
celeba attgan
celeba gdwct
ffhq stylegan
ffhq stylegan2


In [5]:
results=[]

# results/adversarial_deepfakes/ 

In [6]:
df=pd.read_csv('results/adversarial_deepfakes/data9.txt', index_col=0, header=None)
df['probability']=df[1]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [7]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))

correct      
             mean count
dataset                
celeba   0.715475  5648
stargan  0.276558  5648

correct      
             mean count
dataset                
attgan   0.278435  6005
celeba   0.722065  6005

correct      
             mean count
dataset                
celeba   0.711613  3367
gdwct    0.281556  3367

correct      
             mean count
dataset                
ffhq      0.71075  4000
stylegan  0.27700  4000

correct      
              mean count
dataset                 
ffhq       0.71075  4000
stylegan2  0.27475  4000

In [8]:
model="adversarial_deepfakes"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)

adversarial_deepfakes celeba stargan 0.5021246458923513 0.5028308392401433 1.1725256541182034
adversarial_deepfakes celeba attgan 0.49925062447960034 0.49902165902547213 1.1516914694588705
adversarial_deepfakes celeba gdwct 0.5034155034155035 0.500941853481206 1.114947844783051
adversarial_deepfakes ffhq stylegan 0.495375 0.5021558125 1.1563525345374388
adversarial_deepfakes ffhq stylegan2 0.496 0.4969171875 1.1456786093335658


# results/CNNDetection/ 

In [9]:
df=pd.read_csv('results/CNNDetection/data9.txt', index_col=0, header=None)
df['probability']=df[1]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [10]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))

correct      
             mean count
dataset                
celeba   0.000531  5648
stargan  0.382082  5648

correct      
             mean count
dataset                
attgan   0.843131  6005
celeba   0.001332  6005

correct      
             mean count
dataset                
celeba   0.000297  3367
gdwct    0.996139  3367

correct      
             mean count
dataset                
ffhq      0.00075  4000
stylegan  0.98250  4000

correct      
              mean count
dataset                 
ffhq       0.00075  4000
stylegan2  0.91300  4000

In [11]:
model="CNNDetection"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)

CNNDetection celeba stargan 0.19130665722379603 0.0319421023962956 20.619586463977598
CNNDetection celeba attgan 0.4218984179850125 0.16100066486365447 17.02190540224137
CNNDetection celeba gdwct 0.49851499851499853 0.4521608998888476 16.761461515592757
CNNDetection ffhq stylegan 0.492625 0.3487954999999999 16.731231468871776
CNNDetection ffhq stylegan2 0.45425 0.27626784374999996 16.919475290989823


# results/patch-forensics/

In [12]:
df=pd.read_csv('results/patch-forensics/data9.txt', index_col=0, header=None)
df['probability']=df[1]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [13]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))

correct      
             mean count
dataset                
celeba   0.110836  5648
stargan  0.024788  5648

correct      
             mean count
dataset                
attgan   0.008326  6005
celeba   0.104746  6005

correct      
             mean count
dataset                
celeba   0.108108  3367
gdwct    0.307989  3367

correct      
            mean count
dataset               
ffhq           0  4000
stylegan       1  4000

correct      
             mean count
dataset                
ffhq            0  4000
stylegan2       1  4000

In [14]:
model="patch-forensics"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)

patch-forensics celeba stargan 0.06223441926345609 0.009624731159065552 1.7111761031306514
patch-forensics celeba attgan 0.054621149042464616 0.004531097801512899 1.822355084149315
patch-forensics celeba gdwct 0.2044847044847045 0.10313249309388303 1.1390516934312291
patch-forensics ffhq stylegan 0.5 0.026948281249999994 3.2362074689583595
patch-forensics ffhq stylegan2 0.5 0.19223571875 3.204329616877045


# results/MesoNet/ 

In [15]:
df=pd.read_csv('results/MesoNet/data9.txt', index_col=0, header=None)
df['probability']=df[1]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [16]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="MesoNet"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.904214  5648
stargan  0.070822  5648

correct      
             mean count
dataset                
attgan   0.071940  6005
celeba   0.908909  6005

correct      
             mean count
dataset                
celeba   0.906148  3367
gdwct    0.094743  3367

correct      
            mean count
dataset               
ffhq      0.8735  4000
stylegan  0.0390  4000

correct      
             mean count
dataset                
ffhq       0.8735  4000
stylegan2  0.0495  4000

MesoNet celeba stargan 0.488314447592068 0.6635233761205049 1.5110787372049308
MesoNet celeba attgan 0.48767693588676103 0.6745704835201862 1.4773950913156835
MesoNet celeba gdwct 0.5008910008910009 0.5883379177112471 1.911793825871283
MesoNet ffhq stylegan 0.455875 0.45309359375 1.8144420756732618
MesoNet ffhq stylegan2 0.460125 0.48877318750000004 1.7013556059555766


# results/ClassNSeg/ 

In [17]:
df=pd.read_csv('results/ClassNSeg/data9.txt', index_col=0, header=None, usecols=[0,2])
df['probability']=df[2]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [18]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="ClassNSeg"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.745574  5648
stargan  0.178824  5648

correct      
             mean count
dataset                
attgan   0.099750  6005
celeba   0.747877  6005

correct      
             mean count
dataset                
celeba   0.754084  3367
gdwct    0.164835  3367

correct      
            mean count
dataset               
ffhq      0.9310  4000
stylegan  0.0105  4000

correct      
              mean count
dataset                 
ffhq       0.93100  4000
stylegan2  0.05575  4000

ClassNSeg celeba stargan 0.46839589235127477 0.4479793575554334 0.6947851884910289
ClassNSeg celeba attgan 0.4275603663613655 0.4337256976388674 0.6972226587137927
ClassNSeg celeba gdwct 0.4575289575289575 0.4406743450402494 0.6961314610924267
ClassNSeg ffhq stylegan 0.47 0.27624690625000003 0.8280808481553646
ClassNSeg ffhq stylegan2 0.4925 0.28105425 0.8183981901706281


# results/Capsule-Forensics-v2/ 

In [19]:
df=pd.read_csv('results/Capsule-Forensics-v2/data9.txt', index_col=0, header=None)
df['probability']=df[2]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [20]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="Capsule-Forensics-v2"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.930241  5648
stargan  0.087996  5648

correct      
             mean count
dataset                
attgan   0.134221  6005
celeba   0.925562  6005

correct      
             mean count
dataset                
celeba   0.923671  3367
gdwct    0.037422  3367

correct      
            mean count
dataset               
ffhq      0.2285  4000
stylegan  0.9450  4000

correct      
             mean count
dataset                
ffhq       0.2285  4000
stylegan2  0.9665  4000

Capsule-Forensics-v2 celeba stargan 0.5074362606232294 0.6033760321034195 0.716024600773447
Capsule-Forensics-v2 celeba attgan 0.5297252289758535 0.6780682487047638 0.705296251435719
Capsule-Forensics-v2 celeba gdwct 0.4829224829224829 0.407833759927612 0.7274640622530772
Capsule-Forensics-v2 ffhq stylegan 0.587625 0.6018515 0.6789479716392516
Capsule-Forensics-v2 ffhq stylegan2 0.595375 0.6407846875 0.6752816770911358


# Exploiting-Visual-Artifacts/dfgc2021_face2face

In [21]:
df=pd.read_csv('results/Exploiting-Visual-Artifacts/data9_face2face/scores.csv', index_col=0)
df['probability']=df['Score_MLP']
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [22]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="Exploiting-Visual-Artifacts/data9_face2face"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.012925  5648
stargan  0.979816  5648

correct      
             mean count
dataset                
attgan   0.984180  6005
celeba   0.014488  6005

correct      
             mean count
dataset                
celeba   0.015741  3367
gdwct    0.985150  3367

correct      
             mean count
dataset                
ffhq      0.10200  4000
stylegan  0.84775  4000

correct      
             mean count
dataset                
ffhq        0.102  4000
stylegan2   0.955  4000

Exploiting-Visual-Artifacts/data9_face2face celeba stargan 0.49743271954674223 0.5133110118450513 11.096680447412979
Exploiting-Visual-Artifacts/data9_face2face celeba attgan 0.5001665278934222 0.537050542810217 11.138113905131506
Exploiting-Visual-Artifacts/data9_face2face celeba gdwct 0.5005940005940006 0.5333823658741983 10.942288382163655
Exploiting-Visual-Artifacts/data9_face2face ffhq stylegan 0.471375 0.41985975 1.084638064930158
Exploiting-Visual-Artifacts/data9_face2face ffhq stylegan2 0.529375 0.650730875 0.9854603515663285


# Exploiting-Visual-Artifacts/dfgc2021_deepfake

In [23]:
df=pd.read_csv('results/Exploiting-Visual-Artifacts/data9_deepfake/scores.csv', index_col=0)
df['probability']=df['Score_MLP']
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [24]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="Exploiting-Visual-Artifacts/data9_deepfake"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.024788  5648
stargan  0.975921  5648

correct      
             mean count
dataset                
attgan   0.983347  6005
celeba   0.025146  6005

correct      
             mean count
dataset                
celeba   0.023760  3367
gdwct    0.980992  3367

correct      
            mean count
dataset               
ffhq      0.0720  4000
stylegan  0.9525  4000

correct      
             mean count
dataset                
ffhq       0.0720  4000
stylegan2  0.9225  4000

Exploiting-Visual-Artifacts/data9_deepfake celeba stargan 0.49796388101983 0.4999099527070678 16.503103586596943
Exploiting-Visual-Artifacts/data9_deepfake celeba attgan 0.504496253122398 0.5077408848163582 16.485592962786882
Exploiting-Visual-Artifacts/data9_deepfake celeba gdwct 0.5050490050490051 0.5102016999848897 16.392348673788913
Exploiting-Visual-Artifacts/data9_deepfake ffhq stylegan 0.50775 0.4374693125 13.492165853232871
Exploiting-Visual-Artifacts/data9_deepfake ffhq stylegan2 0.50125 0.45394525 13.502642741170478


# CVPRW2019_Face_Artifacts

In [25]:
with open('results/CVPRW2019_Face_Artifacts/data9.json') as f:
    j=json.load(f)
df=pd.DataFrame(j).set_index('file_name')
#df.columns=['headpose_forensic_'+x for x in df.columns]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [26]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="CVPRW2019_Face_Artifacts"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.006728  5648
stargan  0.988669  5648

correct      
             mean count
dataset                
attgan   0.999001  6005
celeba   0.005329  6005

correct      
             mean count
dataset                
celeba   0.007128  3367
gdwct    0.959905  3367

correct      
            mean count
dataset               
ffhq      0.0125  4000
stylegan  0.9945  4000

correct      
             mean count
dataset                
ffhq       0.0125  4000
stylegan2  1.0000  4000

CVPRW2019_Face_Artifacts celeba stargan 0.49822946175637395 0.5153461120133779 11.352717344985212
CVPRW2019_Face_Artifacts celeba attgan 0.5020815986677769 0.5627698538755865 11.401689968676365
CVPRW2019_Face_Artifacts celeba gdwct 0.4838134838134838 0.49399070575191745 11.717470791930197
CVPRW2019_Face_Artifacts ffhq stylegan 0.502875 0.45205418750000004 3.00710519343136
CVPRW2019_Face_Artifacts ffhq stylegan2 0.50625 0.784732 2.994681587027046


# headpose_forensic

In [27]:
with open('results/headpose_forensic/data9.json') as f:
    j=json.load(f)
df=pd.DataFrame(j).set_index('file_name')
#df.columns=['headpose_forensic_'+x for x in df.columns]
df['label']=df.index.map(lambda x: int('ffhq' in x or 'celeba' in x ))
df['pred']= (df['probability']>0.5).astype(np.int)

In [28]:
df['dataset']=df.index.map(lambda u: u.split('/')[-2])
df['correct']=(df['label']==df['pred']).astype(np.int)

In [29]:
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    display(tmpdf.groupby('dataset')[['correct']].aggregate(['mean','count']))
model="headpose_forensic"
for real,fake in pairs:
    tmpdf=df[df['dataset'].isin([real,fake])]
    minlen=tmpdf['dataset'].value_counts().min()
    dfreal=tmpdf[tmpdf['dataset']==real].sample(minlen)
    dffake=tmpdf[tmpdf['dataset']==fake].sample(minlen)
    tmpdf=pd.concat([dfreal,dffake])
    auc=roc_auc_score(tmpdf['label'],tmpdf['probability'])
    acc=tmpdf['correct'].mean()
    loss=log_loss(tmpdf['label'],tmpdf['probability'])
    results.append([model,real,fake,acc,auc,loss])
    print(model,real,fake,acc,auc,loss)    

correct      
             mean count
dataset                
celeba   0.050106  5648
stargan  0.949540  5648

correct      
             mean count
dataset                
attgan   0.965529  6005
celeba   0.053122  6005

correct      
             mean count
dataset                
celeba   0.059697  3367
gdwct    0.956935  3367

correct      
             mean count
dataset                
ffhq      0.20950  4000
stylegan  0.83125  4000

correct      
              mean count
dataset                 
ffhq       0.20950  4000
stylegan2  0.83775  4000

headpose_forensic celeba stargan 0.49955736543909346 0.5100078671083148 11.008132246973569
headpose_forensic celeba attgan 0.5094920899250625 0.5438333306757275 10.972573265853987
headpose_forensic celeba gdwct 0.5031185031185031 0.52127618566585 11.20466672265707
headpose_forensic ffhq stylegan 0.51975 0.48739706250000003 1.2002620828218733
headpose_forensic ffhq stylegan2 0.5215 0.4421743125 1.203472054190763


In [30]:
resultsdf=pd.DataFrame(results,columns=['model','real','fake','accuracy','roc_auc_score','log_loss'])

In [31]:
resultsdf.to_csv('data9_binary_score.csv',index=None)

In [32]:
set(resultsdf['model'])

{'CNNDetection',
 'CVPRW2019_Face_Artifacts',
 'Capsule-Forensics-v2',
 'ClassNSeg',
 'Exploiting-Visual-Artifacts/data9_deepfake',
 'Exploiting-Visual-Artifacts/data9_face2face',
 'MesoNet',
 'adversarial_deepfakes',
 'headpose_forensic',
 'patch-forensics'}

In [33]:
resultsdf

,model,real,fake,accuracy,roc_auc_score,log_loss
0,adversarial_deepfakes,celeba,stargan,0.502125,0.502831,1.172526
1,adversarial_deepfakes,celeba,attgan,0.499251,0.499022,1.151691
2,adversarial_deepfakes,celeba,gdwct,0.503416,0.500942,1.114948
3,adversarial_deepfakes,ffhq,stylegan,0.495375,0.502156,1.156353
4,adversarial_deepfakes,ffhq,stylegan2,0.496000,0.496917,1.145679
5,CNNDetection,celeba,stargan,0.191307,0.031942,20.619586
6,CNNDetection,celeba,attgan,0.421898,0.161001,17.021905
7,CNNDetection,celeba,gdwct,0.498515,0.452161,16.761462
8,CNNDetection,ffhq,stylegan,0.492625,0.348795,16.731231
9,CNNDetection,ffhq,stylegan2,0.454250,0.276268,16.919475


In [34]:
adf=resultsdf.drop('real',axis=1).groupby(['model','fake']).mean().reset_index()
display(adf.sort_values('accuracy'))
display(adf.sort_values('roc_auc_score'))
display(adf.sort_values('log_loss'))

,model,fake,accuracy,roc_auc_score,log_loss
45,patch-forensics,attgan,0.054621,0.004531,1.822355
47,patch-forensics,stargan,0.062234,0.009625,1.711176
2,CNNDetection,stargan,0.191307,0.031942,20.619586
46,patch-forensics,gdwct,0.204485,0.103132,1.139052
0,CNNDetection,attgan,0.421898,0.161001,17.021905
15,ClassNSeg,attgan,0.427560,0.433726,0.697223
4,CNNDetection,stylegan2,0.454250,0.276268,16.919475
33,MesoNet,stylegan,0.455875,0.453094,1.814442
16,ClassNSeg,gdwct,0.457529,0.440674,0.696131
34,MesoNet,stylegan2,0.460125,0.488773,1.701356


,model,fake,accuracy,roc_auc_score,log_loss
45,patch-forensics,attgan,0.054621,0.004531,1.822355
47,patch-forensics,stargan,0.062234,0.009625,1.711176
48,patch-forensics,stylegan,0.500000,0.026948,3.236207
2,CNNDetection,stargan,0.191307,0.031942,20.619586
46,patch-forensics,gdwct,0.204485,0.103132,1.139052
0,CNNDetection,attgan,0.421898,0.161001,17.021905
49,patch-forensics,stylegan2,0.500000,0.192236,3.204330
18,ClassNSeg,stylegan,0.470000,0.276247,0.828081
4,CNNDetection,stylegan2,0.454250,0.276268,16.919475
19,ClassNSeg,stylegan2,0.492500,0.281054,0.818398


,model,fake,accuracy,roc_auc_score,log_loss
14,Capsule-Forensics-v2,stylegan2,0.595375,0.640785,0.675282
13,Capsule-Forensics-v2,stylegan,0.587625,0.601851,0.678948
17,ClassNSeg,stargan,0.468396,0.447979,0.694785
16,ClassNSeg,gdwct,0.457529,0.440674,0.696131
15,ClassNSeg,attgan,0.427560,0.433726,0.697223
10,Capsule-Forensics-v2,attgan,0.529725,0.678068,0.705296
12,Capsule-Forensics-v2,stargan,0.507436,0.603376,0.716025
11,Capsule-Forensics-v2,gdwct,0.482922,0.407834,0.727464
19,ClassNSeg,stylegan2,0.492500,0.281054,0.818398
18,ClassNSeg,stylegan,0.470000,0.276247,0.828081


In [35]:
adf=resultsdf.drop(['real','fake'],axis=1).groupby(['model']).mean().reset_index()
display(adf.sort_values('accuracy'))
display(adf.sort_values('roc_auc_score'))
display(adf.sort_values('log_loss'))

,model,accuracy,roc_auc_score,log_loss
9,patch-forensics,0.264268,0.067294,2.222624
0,CNNDetection,0.411719,0.254033,17.610732
3,ClassNSeg,0.463197,0.375936,0.746924
6,MesoNet,0.478576,0.573660,1.683213
1,CVPRW2019_Face_Artifacts,0.498650,0.561779,8.094733
7,adversarial_deepfakes,0.499233,0.500373,1.148239
5,Exploiting-Visual-Artifacts/data9_face2face,0.499789,0.530867,7.049436
4,Exploiting-Visual-Artifacts/data9_deepfake,0.503302,0.481853,15.275171
8,headpose_forensic,0.510684,0.500938,7.117821
2,Capsule-Forensics-v2,0.540617,0.586383,0.700603


,model,accuracy,roc_auc_score,log_loss
9,patch-forensics,0.264268,0.067294,2.222624
0,CNNDetection,0.411719,0.254033,17.610732
3,ClassNSeg,0.463197,0.375936,0.746924
4,Exploiting-Visual-Artifacts/data9_deepfake,0.503302,0.481853,15.275171
7,adversarial_deepfakes,0.499233,0.500373,1.148239
8,headpose_forensic,0.510684,0.500938,7.117821
5,Exploiting-Visual-Artifacts/data9_face2face,0.499789,0.530867,7.049436
1,CVPRW2019_Face_Artifacts,0.498650,0.561779,8.094733
6,MesoNet,0.478576,0.573660,1.683213
2,Capsule-Forensics-v2,0.540617,0.586383,0.700603


,model,accuracy,roc_auc_score,log_loss
2,Capsule-Forensics-v2,0.540617,0.586383,0.700603
3,ClassNSeg,0.463197,0.375936,0.746924
7,adversarial_deepfakes,0.499233,0.500373,1.148239
6,MesoNet,0.478576,0.573660,1.683213
9,patch-forensics,0.264268,0.067294,2.222624
5,Exploiting-Visual-Artifacts/data9_face2face,0.499789,0.530867,7.049436
8,headpose_forensic,0.510684,0.500938,7.117821
1,CVPRW2019_Face_Artifacts,0.498650,0.561779,8.094733
4,Exploiting-Visual-Artifacts/data9_deepfake,0.503302,0.481853,15.275171
0,CNNDetection,0.411719,0.254033,17.610732
